In [1]:
import requests
import json
import warnings
import pandas as pd
from datetime import datetime, timedelta, date
from pprint import pprint

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# OBTENCIÓN DE DATOS DE INTERCAMBIO 

In [2]:
lang          = 'es'

start_date1   = '2024-10-15'
end_date1     = '2024-01-01'

start_date2   = '2023-12-31'
end_date2     = '2023-01-01'

start_date3   = '2022-12-31'
end_date3     = '2022-01-01'

In [3]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [4]:
def intercambio_datos(lang, start_date1, end_date1, start_date2, end_date2, start_date3, end_date3):
    #Intercambio 2024
    query = f"start_date={end_date1}T00:00&end_date={start_date1}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()



    for dato in data['included']:
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)


                fecha =  fecha.strftime("%d/%m/%Y")


                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)



    df_intercambio1 = pd.DataFrame()
    df_intercambio1['nombre']               = lista_nombres
    df_intercambio1['tipo de intercambio']  = lista_tipos
    df_intercambio1['Valores']              = lista_valores
    df_intercambio1["Porcentaje"]           = lista_porcentajes
    df_intercambio1["Fecha actualización"]  = lista_fechas

    #Intercambio 2023
    query = f"start_date={end_date2}T00:00&end_date={start_date2}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()



    for dato in data['included']:
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)


                fecha =  fecha.strftime("%d/%m/%Y")


                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)



    df_intercambio2 = pd.DataFrame()
    df_intercambio2['nombre']               = lista_nombres
    df_intercambio2['tipo de intercambio']  = lista_tipos
    df_intercambio2['Valores']              = lista_valores
    df_intercambio2["Porcentaje"]           = lista_porcentajes
    df_intercambio2["Fecha actualización"]  = lista_fechas

    #Intercambio 2022
    query = f"start_date={end_date3}T00:00&end_date={start_date3}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()



    for dato in data['included']:
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)


                fecha =  fecha.strftime("%d/%m/%Y")


                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)



    df_intercambio3 = pd.DataFrame()
    df_intercambio3['nombre']               = lista_nombres
    df_intercambio3['tipo de intercambio']  = lista_tipos
    df_intercambio3['Valores']              = lista_valores
    df_intercambio3["Porcentaje"]           = lista_porcentajes
    df_intercambio3["Fecha actualización"]  = lista_fechas

    #Concatenación de intercambios de diferentes años
    df_intercambio = pd.concat([df_intercambio3, df_intercambio2, df_intercambio1], ignore_index =True)

    df_intercambio['tipo de intercambio'] = df_intercambio['tipo de intercambio'].replace('saldo', 'Saldo')

    return df_intercambio

df_intercambio = intercambio_datos(lang, start_date1, end_date1, start_date2, end_date2, start_date3, end_date3)


df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-2873.3,0.040691,01/01/2022
1,Francia,Exportación,-2674.3,0.033530,02/01/2022
2,Francia,Exportación,-6444.3,0.078966,03/01/2022
3,Francia,Exportación,-41761.7,0.727676,04/01/2022
4,Francia,Exportación,-42497.1,0.801499,05/01/2022
...,...,...,...,...,...
10531,Andorra,Saldo,-24.0,1.000000,11/10/2024
10532,Andorra,Saldo,-159.0,1.000000,12/10/2024
10533,Andorra,Saldo,-369.0,1.000000,13/10/2024
10534,Andorra,Saldo,-68.0,1.000000,14/10/2024


In [5]:
df_intercambio.to_csv('intercambio_electrico.csv')

In [ ]:
fig = px.bar(df_intercambio, x = 'nombre', y = 'Valores', color = 'tipo de intercambio',
             title= 'Valor del intercambio por País y Tipo',
             labels= {'Valores' : 'Valor de intercambio', 'nombre' : 'nombre'},
             barmode= 'group')

fig.show()

In [ ]:
fig = px.scatter(df_intercambio, x = 'Valores', y = 'Porcentaje', color = 'nombre',
                 size= 'Porcentaje', hover_name= 'tipo de intercambio',
                 title = 'Relación entre el valor del intercambio y el porcentaje de cambio',
                 size_max= 10)

fig.show()

In [ ]:
fig = px.line(df_intercambio, x = 'Fecha actualización', y = 'Valores', color= 'nombre',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por País y Tipo',
              line_dash='tipo de intercambio',
              markers= True)

fig.show()

In [ ]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Francia'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio Francia',
              line_dash='tipo de intercambio',
              markers= True)

fig.show()

In [ ]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Portugal'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Portugal',
              line_dash= 'tipo de intercambio',
              markers= True)

fig.show()

In [ ]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Marruecos'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Marruecos',
              line_dash= 'tipo de intercambio',
              markers= True)

fig.show()

In [ ]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Andorra'],
              x='Fecha actualización', y='Valores', color='tipo de intercambio',
              line_group='tipo de intercambio',
              title='Evolución de los valores de intercambio por Andorra',
              markers=True,
              line_dash='tipo de intercambio')

fig.show()
